In [55]:
context = """Instruction tuning large language models (LLMs) using machine-generated
instruction-following data has been shown to improve zero-shot capabilities on
new tasks, but the idea is less explored in the multimodal field. We present the
first attempt to use language-only GPT-4 to generate multimodal language-image
instruction-following data. By instruction tuning on such generated data, we introduce LLaVA: Large Language and Vision Assistant, an end-to-end trained
large multimodal model that connects a vision encoder and an LLM for generalpurpose visual and language understanding. To facilitate future research on visual
instruction following, we construct two evaluation benchmarks with diverse and
challenging application-oriented tasks. Our experiments show that LLaVA demonstrates impressive multimodal chat abilities, sometimes exhibiting the behaviors
of multimodal GPT-4 on unseen images/instructions, and yields a 85.1% relative score compared with GPT-4 on a synthetic multimodal instruction-following
dataset. When fine-tuned on Science QA, the synergy of LLaVA and GPT-4
achieves a new state-of-the-art accuracy of 92.53%. We make GPT-4 generated
visual instruction tuning data, our model, and code publicly available.
1 Introduction
Humans interact with the world through many channels such as vision and language, as each
individual channel has a unique advantage in representing and communicating certain concepts, and
thus facilitates a better understanding of the world. One of the core aspirations in artificial intelligence
is to develop a general-purpose assistant that can effectively follow multi-modal vision-and-language
instructions, aligned with human intent to complete various real-world tasks in the wild [4, 27, 26].
To this end, the community has witnessed an emergent interest in developing language-augmented
foundation vision models [27, 16], with strong capabilities in open-world visual understanding
such as classification [40, 21, 57, 54, 39], detection [29, 62, 33], segmentation [25, 63, 58] and
captioning [50, 28], as well as visual generation and editing [42, 43, 56, 15, 44, 30]. We refer readers
to the Computer Vision in the Wild reading list for a more up-to-date literature compilation [12]. In
this line of work, each task is solved independently by one single large vision model, with the task
instruction implicitly considered in the model design. Further, language is only utilized to describe
the image content. While this allows language to play an important role in mapping visual signals to
language semantics—a common channel for human communication, it leads to models that usually
have a fixed interface with limited interactivity and adaptability to the user’s instructions.
Large language models (LLM), on the other hand, have shown that language can play a wider
role: a universal interface for a general-purpose assistant, where various task instructions can be
explicitly represented in language and guide the end-to-end trained neural assistant to switch to the
task of interest to solve it. For example, the recent success of ChatGPT [35] and GPT-4 [36] have
demonstrated the power of aligned LLMs in following human instructions, and have stimulated
tremendous interest in developing open-source LLMs. Among them, LLaMA [49] is an opensource LLM that matches the performance of GPT-3. Alpaca [48], Vicuna [9], GPT-4-LLM [38]
37th Conference on Neural Information Processing Systems (NeurIPS 2023).
arXiv:2304.08485v2 [cs.CV] 11 Dec 2023
utilize various machine-generated high-quality instruction-following samples to improve the LLM’s
alignment ability, reporting impressive performance compared with proprietary LLMs. Importantly,
this line of work is text-only.
In this paper, we present visual instruction-tuning, the first attempt to extend instruction-tuning to
the language-image multimodal space, to pave the way towards building a general-purpose visual
assistant. In particular, our paper makes the following contributions:
• Multimodal instruction-following data. One key challenge is the lack of vision-language
instruction-following data. We present a data reformation perspective and pipeline to convert
image-text pairs into an appropriate instruction-following format, using ChatGPT/GPT-4.
• Large multimodal models. We develop a large multimodal model (LMM), by connecting the
open-set visual encoder of CLIP [40] with the language decoder Vicuna [9], and fine-tuning
end-to-end on our generated instructional vision-language data. Our empirical study validates
the effectiveness of using generated data for LMM instruction-tuning, and suggests practical
tips for building a general-purpose instruction-following visual agent. When ensembled with
GPT-4, our approach achieves SoTA on the Science QA [34] multimodal reasoning dataset.
• Multimodal instruction-following benchmark. We present LLaVA-Bench with two challenging
benchmarks, with a diverse selection of paired images, instructions and detailed annotations.
• Open-source. We release the following assets to the public: the generated multimodal instruction
data, the codebase, the model checkpoints, and a visual chat demo.
2 Related Work
Multimodal Instruction-following Agents. In computer vision, existing works that build instructionfollowing agents can be broadly categorized into two classes: (i) End-to-end trained models, which
are separately explored for each specific research topic. For example, the vision-language navigation
task [3, 19] and Habitat [47] require the embodied AI agent to follow natural language instructions
and take a sequence of actions to complete goals in visual environments. In the image editing domain,
given an input image and a written instruction that tells the agent what to do, InstructPix2Pix [6]
edits images by following the human instructions. (ii) A system that coordinates various models
via LangChain [1] / LLMs [35], such as Visual ChatGPT [53], X-GPT [63], MM-REACT [55],
VisProg [18], and ViperGPT [46]. While sharing the same goal in building instruction-following
agents, we focus on developing an end-to-end trained language-vision multimodal model for multiple
tasks.
Instruction Tuning. In the natural language processing (NLP) community, to enable LLMs such
as GPT-3 [7], T5 [41], PaLM [10], and OPT [60] to follow natural language instructions and
complete real-world tasks, researchers have explored methods for LLM instruction-tuning [37, 52, 51],
leading to instruction-tuned counterparts such as InstructGPT [37]/ChatGPT [35], FLAN-T5 [11],
FLAN-PaLM [11], and OPT-IML [22], respectively. It turns out that this simple approach can
effectively improve the zero- and few-shot generalization abilities of LLMs. It is thus natural
to borrow the idea from NLP to computer vision. More broadly, the teacher-student distillation
ideas with foundation models have been studied in other topics such as image classification [14].
Flamingo [2] can be viewed as the GPT-3 moment in the multimodal domain, due to its strong
performance on zero-shot task transfer and in-context-learning. Other LMMs trained on imagetext pairs include BLIP-2 [28], FROMAGe [24], and KOSMOS-1 [20]. PaLM-E [13] is an LMM
for embodied AI. Based on the recent “best” open-source LLM LLaMA, OpenFlamingo [5] and
LLaMA-Adapter [59] are open-source efforts that enable LLaMA to use image inputs, paving the
way to build open-source multimodal LLMs. While these models present promising task transfer
generalization performance, they are not explicitly tuned with vision-language instruction data, and
their performance in multimodal tasks usually falls short compared to language-only tasks. In this
paper, we aim to fill this gap and study its effectiveness. Finally, note that visual instruction tuning
is different from visual prompt tuning [23]: the former aims to improve the model’s instructionfollowing abilities, while the latter aims to improve the parameter-efficiency in model adaptation.
3 GPT-assisted Visual Instruction Data Generation
The community has witnessed a surge in the amount of public multimodal data such as image-text
pairs, ranging from CC [8] to LAION [45]. However, when it comes to multimodal instruction2
Context type 1: Captions
A group of people standing outside of a black vehicle with various luggage.
Luggage surrounds a vehicle in an underground parking area
People try to fit all of their luggage in an SUV.
The sport utility vehicle is parked in the public garage, being packed for a trip
Some people with luggage near a van that is transporting it.
Context type 2: Boxes
person: [0.681, 0.242, 0.774, 0.694], backpack: [0.384, 0.696, 0.485, 0.914], suitcase: ...<omitted>
Response type 1: conversation
Question: What type of vehicle is featured in the image?
Answer: The image features a black sport utility vehicle (SUV) ...<omitted>
Response type 2: detailed description
The image is an underground parking area with a black sport utility vehicle (SUV) parked. There
are three people in the scene, with one person standing closer to the left side of the vehicle, another
person in the middle, and the third person on the right side. They are all working together to pack
their luggage into the SUV for a trip. ...<omitted>
Response type 3: complex reasoning
Question: What challenges do these people face?
Answer: In the image, a group of people is standing outside a black SUV in a parking area,
surrounded by various pieces of luggage, including suitcases and backpacks. They are facing the
challenge of fitting all their luggage into the black SUV. There are multiple suitcases and backpacks
to be packed, which suggests that the group has a significant amount of belongings ...<omitted>
Table 1: One example to illustrate the instruction-following data. The top block shows the contexts
such as captions and boxes used to prompt GPT, and the bottom block shows the three types of
responses. Note that the visual image is not used to prompt GPT, we only show it here as a reference.
following data, the available amount is limited, partially because the process for creating such data is
time-consuming and less well-defined when human crowd-scouring is considered. Inspired by the
success of recent GPT models in text-annotation tasks [17], we propose to leverage ChatGPT/GPT-4
for multimodal instruction-following data collection, based on the widely existing image-pair data.
For an image Xv and its associated caption Xc, it is natural to create a set of questions Xq with the
intent to instruct the assistant to describe the image content. We prompt GPT-4 to curate such a list
of questions (see details in Appendix). Therefore, a simple way to expand an image-text pair to its
instruction-following version is Human : Xq Xv<STOP> Assistant : Xc<STOP>. Though cheap to
construct, this simple expanded version lacks diversity and in-depth reasoning in both the instructions
and responses.
To mitigate this issue, we leverage language-only GPT-4 or ChatGPT as the strong teacher (both
accept only text as input), to create instruction-following data involving visual content. Specifically,
in order to encode an image into its visual features to prompt a text-only GPT, we use two types of
symbolic representations: (i) Captions typically describe the visual scene from various perspectives;
(ii) Bounding boxes usually localize the objects in the scene, and each box encodes the object concept
and its spatial location. One example is shown in the top block of Table 14.
This symbolic representation allows us to encode the image as an LLM-recognizable sequence. We
use COCO images [31] and generate three types of instruction-following data. One example per type
is shown in the bottom block of Table 14. For each type, we first manually design a few examples.
They are the only human annotations we have during data collection, and are used as seed examples
in in-context-learning to query GPT-4.
• Conversation. We design a conversation between the assistant and a person asking questions
about this photo. The answers are in a tone as if the assistant is seeing the image and answering
the question. A diverse set of questions are asked about the visual content of the image, including
the object types, counting the objects, object actions, object locations, relative positions between
objects. Only questions that have definite answers are considered. Please see Appendix for the
detailed prompt.
• Detailed description. To include a rich and comprehensive description for an image, we create a
list of questions with such an intent. We prompt GPT-4 then curate the list (see detailed prompts
3
and curation process in Appendix). For each image, we randomly sample one question from the
list to ask GPT-4 to generate the detailed description.
• Complex reasoning. The above two types focus on the visual content itself, based on which
we further create in-depth reasoning questions. The answers typically require a step-by-step
reasoning process by following rigorous logic.
We collect 158K unique language-image instruction-following samples in total, including 58K in
conversations, 23K in detailed description, and 77k in complex reasoning, respectively. We ablated
the use of ChatGPT and GPT-4 in our early experiments, and found that GPT-4 consistently provides
higher quality instruction-following data, such as spatial reasoning.
4 Visual Instruction Tuning
4.1 Architecture
The primary goal is to effectively leverage the capabilities of both the pre-trained LLM and visual
model. The network archtecture is illustrated in Figure 1. We choose Vicuna [9] as our LLM fϕ(·)
parameterized by ϕ, as it has the best instruction following capabilities in language tasks among"""

In [56]:
context = context.split('.')
len(context)

119

In [51]:
import runpod

runpod.api_key = "YOUR_RUNPOD_API_KEY"

endpoint = runpod.Endpoint("ENDPOINT_ID")

run_request = endpoint.run(
    {"input": {
        "context": context,
        "instruction": "You are a question answering bot who uses the provided context to answer a question\n",
        "question": "<question>What abilities does LLaVa demonstrate?</question>",
        "target_tokens": 300,
    }
})

In [52]:
%%time 
print(run_request.status())
response = run_request.output(timeout=60)
print(response)

IN_QUEUE
{'compressed_prompt': 'You are a question answering bot who uses the provided context to answer a question\n\n\n By instruction tuning on such generated data, we introduce LLaVA: Large Language and Vision Assistant, anend trained\nlargeal model that connects a visionoder and an LLM for generalpurpose visual and language understanding\n\n Our experiments thatVA demonstrates impressiveal chat abilities, sometimes exhibiting the behaviors\nofalPT-4 on unseen images/ions, and yields a 85\n\n Whentuned on Science QA, the synergy ofVA and4\nieves a newart accuracy of 92\n\n We make4 generated\nvisual instruction tuning data, our model, and code available\n\n One the core aspirations in intelligence\nis to develop a generalpurpose assistant that can effectively follow-al vision--languageions, with human intent to complete various tasks in the wild [4, 27, 26]\n\n We refer readersto the Vision in the Wild reading list for a moredate literature compilation [12]\n\n Further, language is

In [53]:
from pprint import pprint
pprint(response)

{'compressed_prompt': 'You are a question answering bot who uses the provided '
                      'context to answer a question\n'
                      '\n'
                      '\n'
                      ' By instruction tuning on such generated data, we '
                      'introduce LLaVA: Large Language and Vision Assistant, '
                      'anend trained\n'
                      'largeal model that connects a visionoder and an LLM for '
                      'generalpurpose visual and language understanding\n'
                      '\n'
                      ' Our experiments thatVA demonstrates impressiveal chat '
                      'abilities, sometimes exhibiting the behaviors\n'
                      'ofalPT-4 on unseen images/ions, and yields a 85\n'
                      '\n'
                      ' Whentuned on Science QA, the synergy ofVA and4\n'
                      'ieves a newart accuracy of 92\n'
                      '\n'
                      ' 